[Amirhossein Mahmoudi](https://ammahmoudi.github.io)
# Adams–Bashforth Method with RK4 initializer
solution to [Numerical Analysis (R. Burden, Aires, A. Burden) [10th Ed.]](https://www.amazon.com/Richard-Burden-Numerical-Analysis-Hardcover/dp/B00SB3UL20) Section 5.6 question 3.b

![5_6_3_b](images/5_6_3_b.png)

In [1]:
import pandas as pd
from math import *
import numpy as np

## Corrector Values for AB Method 2,3,4,5 step

In [2]:
correctors = [
	[2, 3, -1],
	[12, 23, -16, 5],
	[24, 55, -59, 37, -9],
	[720, 1901, -2774, 2616, -1274, 251]
]

## RK-4 Method

In [3]:
def rk4(f, t, y,h):
	k1 = h*f(t, y)
	k2 = h*f(t + h/2, y + k1/2)
	k3 = h*f(t + h/2, y + k2/2)
	k4 = h*f(t + h, y + k3)
	return y + (k1 + 2*k2 + 2*k3 + k4)/6

## Adams–Bashforth method with rk4 initializer

In [4]:
def abN_rk4(f, f_actual, t, y, p, h, N,correctors):
	#coeffients
	corrector = correctors[N-2]
	#number of steps
	n = int((p-t)/h)
	#intialize time array
	time = [t+h*i for i in range(n+1)]
	#initialze y_vaules
	y_values = [y]
	#make inital values from rk4
	for i in range(1,N):
		y_values.append(rk4(f, time[i-1], y_values[i-1],h))
	#run the algorithm
	for i in range(N, n+1):
		fix = 0
		for j in range(N):
			fix += corrector[j+1] * f(time[i-j-1], y_values[i-j-1])
		yn = y_values[i-1] + h/corrector[0]*fix

		y_values.append(yn)
	y_actual=np.ones_like(time)
	for i in range(len(y_values)):
		y_actual[i]=f_actual(time[i])

	return np.c_[time,y_values,y_actual]


# Test

In [5]:
f = lambda t, y: 1+y/t+(y/t)**2

f_actual = lambda t: t*tan(log(t))

t = 1
y = 0
p = 3
h = 0.2

result=pd.DataFrame()
for i in range(2,6):
    result_n=abN_rk4(f, f_actual, t, y, p, h, i,correctors)
    if i==2:
        result=pd.DataFrame(result_n[:,:2],columns=['time','2-step'])
    else:
        result[str(i)+"-step"]=pd.Series(result_n[:,1])
result['exact']=pd.Series(result_n[:,2])
result.head(n=12)
    
    

,time,2-step,3-step,4-step,5-step,exact
0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,1.2,0.221246,0.221246,0.221246,0.221246,0.221243
2,1.4,0.486755,0.489684,0.489684,0.489684,0.489682
3,1.6,0.805488,0.812432,0.812752,0.812752,0.812753
4,1.8,1.185693,1.198211,1.199042,1.199432,1.199439
5,2.0,1.637794,1.658431,1.660306,1.661318,1.661282
6,2.2,2.175379,2.207999,2.211745,2.213479,2.213502
7,2.4,2.816395,2.866767,2.873532,2.876278,2.876551
8,2.6,3.584918,3.661748,3.673327,3.677724,3.678475
9,2.8,4.513842,4.630528,4.649894,4.657003,4.658665
